<a href="https://colab.research.google.com/github/D13G04L3X/Librerias-dinamicas/blob/main/Entrega_Final_Algoritmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get -qq update
!DEBIAN_FRONTEND=noninteractive apt-get -qq install swig build-essential python3-dev

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
%%bash
set -e

# -------- HMM.hpp --------
cat > HMM.hpp <<'EOF'
#pragma once
#include <string>
#include <array>
#include <vector>

struct HMM {
    // Estados: 0=H, 1=L
    std::array<double,2> pi{0.5, 0.5};                  // Inicio -> H/L
    std::array<std::array<double,2>,2> A{{               // Transiciones
        std::array<double,2>{0.5, 0.5},                  // H->H, H->L
        std::array<double,2>{0.4, 0.6}                   // L->H, L->L
    }};
    std::array<std::array<double,4>,2> B{{               // Emisiones por estado
        std::array<double,4>{0.2,0.3,0.3,0.2},           // H: A C G T
        std::array<double,4>{0.3,0.2,0.2,0.3}            // L: A C G T
    }};

    std::array<int,256> map{};
    HMM();

    // Forward (log-space). Retorna log P(seq|modelo).
    double evaluate_log(const std::string& dna) const;

    // Viterbi. Retorna ruta 'H'/'L' del mismo largo que la entrada.
    std::string recognize(const std::string& dna) const;
};
EOF

# -------- HMM.cpp --------
cat > HMM.cpp <<'EOF'
#include "HMM.hpp"
#include <cmath>
#include <stdexcept>

using std::string;
using std::vector;

HMM::HMM() {
    map.fill(-1);
    map['A']=0; map['C']=1; map['G']=2; map['T']=3;
    map['a']=0; map['c']=1; map['g']=2; map['t']=3;
}

static inline double log_sum_exp2(double a, double b){
    if (a == -INFINITY) return b;
    if (b == -INFINITY) return a;
    double m = (a > b) ? a : b;
    return m + std::log2(std::exp2(a-m) + std::exp2(b-m));
}

double HMM::evaluate_log(const string& dna) const {
    if (dna.empty()) return -INFINITY;
    const int N = (int)dna.size();
    vector<std::array<double,2>> dp(N);

    auto eprob = [&](int state, char base)->double{
        int idx = map[(unsigned char)base];
        if (idx < 0) throw std::invalid_argument("Invalid base");
        return std::log2(B[state][idx]);
    };

    // init
    dp[0][0] = std::log2(pi[0]) + eprob(0, dna[0]);
    dp[0][1] = std::log2(pi[1]) + eprob(1, dna[0]);

    // recursion
    for (int t=1; t<N; ++t){
        for (int s=0; s<2; ++s){
            double fromH = dp[t-1][0] + std::log2(A[0][s]);
            double fromL = dp[t-1][1] + std::log2(A[1][s]);
            dp[t][s] = log_sum_exp2(fromH, fromL) + eprob(s, dna[t]);
        }
    }
    return log_sum_exp2(dp[N-1][0], dp[N-1][1]);
}

std::string HMM::recognize(const string& dna) const {
    if (dna.empty()) return "";
    const int N = (int)dna.size();
    vector<std::array<double,2>> v(N);
    vector<std::array<int,2>> back(N);

    auto eprob = [&](int state, char base)->double{
        int idx = map[(unsigned char)base];
        if (idx < 0) throw std::invalid_argument("Invalid base");
        return std::log2(B[state][idx]);
    };

    // init
    v[0][0] = std::log2(pi[0]) + eprob(0, dna[0]);
    v[0][1] = std::log2(pi[1]) + eprob(1, dna[0]);
    back[0] = {-1,-1};

    for (int t=1; t<N; ++t){
        // state H=0
        {
            double fromH = v[t-1][0] + std::log2(A[0][0]);
            double fromL = v[t-1][1] + std::log2(A[1][0]);
            if (fromH >= fromL) { v[t][0]=fromH; back[t][0]=0; }
            else                { v[t][0]=fromL; back[t][0]=1; }
            v[t][0] += eprob(0, dna[t]);
        }
        // state L=1
        {
            double fromH = v[t-1][0] + std::log2(A[0][1]);
            double fromL = v[t-1][1] + std::log2(A[1][1]);
            if (fromH >= fromL) { v[t][1]=fromH; back[t][1]=0; }
            else                { v[t][1]=fromL; back[t][1]=1; }
            v[t][1] += eprob(1, dna[t]);
        }
    }

    int last = (v[N-1][0] >= v[N-1][1]) ? 0 : 1;
    string path(N, 'H');
    for (int t=N-1; t>=0; --t){
        path[t] = (last==0) ? 'H' : 'L';
        last = back[t][last];
        if (t==0) break;
    }
    return path;
}
EOF

# -------- SWIG interface hmm.i --------
cat > hmm.i <<'EOF'
%module hmm
%{
#include "HMM.hpp"
%}

%include "std_string.i"
%include "HMM.hpp"

%inline %{
    std::string Reconocimiento(const std::string& dna) {
        HMM m; return m.recognize(dna);
    }
    double Evaluacion(const std::string& dna) {
        HMM m; return m.evaluate_log(dna);
    }
%}
EOF

# -------- setup.py (setuptools) --------
cat > setup.py <<'EOF'
from setuptools import setup, Extension

hmm_module = Extension(
    '_hmm',
    sources=['HMM.cpp', 'hmm_wrap.cxx'],
)

setup(
    name='hmm',
    version='0.1',
    author='HMM SWIG Example',
    description='HMM (Forward/Viterbi) via SWIG + C++',
    ext_modules=[hmm_module],
    py_modules=['hmm'],
)
EOF

# -------- Implementación Python pura para comparar --------
cat > python_native.py <<'EOF'
import math

PI = [0.5, 0.5]                 # Start -> H, L
A  = [[0.5, 0.5],               # H->H, H->L
      [0.4, 0.6]]               # L->H, L->L
B  = [[0.2,0.3,0.3,0.2],        # H emissions A,C,G,T
      [0.3,0.2,0.2,0.3]]        # L emissions

IDX = {'A':0,'C':1,'G':2,'T':3,'a':0,'c':1,'g':2,'t':3}

def _eprob(s:int, b:str)->float:
    i = IDX.get(b, -1)
    if i < 0: raise ValueError("Invalid base")
    return B[s][i]

def forward_log(seq:str)->float:
    if not seq: return float('-inf')
    n = len(seq)
    dp = [[0.0,0.0] for _ in range(n)]
    def lse(a,b):
        if a==-math.inf: return b
        if b==-math.inf: return a
        m=max(a,b); return m + math.log2(math.exp2(a-m)+math.exp2(b-m))
    dp[0][0] = math.log2(PI[0]) + math.log2(_eprob(0, seq[0]))
    dp[0][1] = math.log2(PI[1]) + math.log2(_eprob(1, seq[0]))
    for t in range(1,n):
        for s in (0,1):
            fromH = dp[t-1][0] + math.log2(A[0][s])
            fromL = dp[t-1][1] + math.log2(A[1][s])
            dp[t][s] = lse(fromH, fromL) + math.log2(_eprob(s, seq[t]))
    return lse(dp[n-1][0], dp[n-1][1])

def viterbi(seq:str)->str:
    if not seq: return ""
    n=len(seq)
    v=[[0.0,0.0] for _ in range(n)]
    back=[[-1,-1] for _ in range(n)]
    v[0][0] = math.log2(PI[0]) + math.log2(_eprob(0, seq[0]))
    v[0][1] = math.log2(PI[1]) + math.log2(_eprob(1, seq[0]))
    for t in range(1,n):
        h_from_h = v[t-1][0] + math.log2(A[0][0])
        h_from_l = v[t-1][1] + math.log2(A[1][0])
        if h_from_h >= h_from_l:
            v[t][0] = h_from_h; back[t][0]=0
        else:
            v[t][0] = h_from_l; back[t][0]=1
        v[t][0] += math.log2(_eprob(0, seq[t]))

        l_from_h = v[t-1][0] + math.log2(A[0][1])
        l_from_l = v[t-1][1] + math.log2(A[1][1])
        if l_from_h >= l_from_l:
            v[t][1] = l_from_h; back[t][1]=0
        else:
            v[t][1] = l_from_l; back[t][1]=1
        v[t][1] += math.log2(_eprob(1, seq[t]))
    last = 0 if v[n-1][0] >= v[n-1][1] else 1
    path=['H']*n
    for t in range(n-1,-1,-1):
        path[t] = 'H' if last==0 else 'L'
        last = back[t][last]
        if t==0: break
    return ''.join(path)
EOF

echo "Archivos creados:"
ls -1


Archivos creados:
build
HMM.cpp
_hmm.cpython-312-x86_64-linux-gnu.so
HMM.hpp
hmm.i
hmm.py
hmm_wrap.cxx
__pycache__
python_native.py
sample_data
setup.py


In [ ]:
!ls

build				      HMM.hpp  hmm_wrap.cxx	 sample_data
HMM.cpp				      hmm.i    __pycache__	 setup.py
_hmm.cpython-312-x86_64-linux-gnu.so  hmm.py   python_native.py


In [ ]:
%%bash
set -e
swig -c++ -python hmm.i
python3 setup.py build_ext --inplace
ls -l | sed -n '1,200p'

running build_ext
building '_hmm' extension
x86_64-linux-gnu-g++ -fno-strict-overflow -Wsign-compare -DNDEBUG -g -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -I/usr/include/python3.12 -c HMM.cpp -o build/temp.linux-x86_64-cpython-312/HMM.o
x86_64-linux-gnu-g++ -fno-strict-overflow -Wsign-compare -DNDEBUG -g -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -I/usr/include/python3.12 -c hmm_wrap.cxx -o build/temp.linux-x86_64-cpython-312/hmm_wrap.o
x86_64-linux-gnu-g++ -fno-strict-overflow -Wsign-compare -DNDEBUG -g -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -g -fwrapv -O2 build/temp.linux-x86_64-cpython-312/HMM.o build/temp.linux-x86_64-cpython-312/hmm_wrap.o -L/usr/lib/x86_64-linux-gnu -o build/lib.linux-x86_64-cpython-312/_hmm.cpython-312-x86_64-linux-gnu.so
copying build/lib.linux-x86_64-c

In [ ]:
import time, math, random
import hmm, python_native as nat

NUC="GGCACTGAA"
def rand_seq(n): return ''.join(random.choice(NUC) for _ in range(n))

def median_time(fn, *args, repeats=7, warmup=1):
    for _ in range(warmup): fn(*args)                 # calentamiento
    ts=[]
    for _ in range(repeats):
        t0=time.perf_counter(); fn(*args); t1=time.perf_counter()
        ts.append(t1-t0)
    ts.sort()
    return ts[len(ts)//2]

for n in (1_000, 5_000, 10_000, 50_000):
    s = rand_seq(n)
    tf_cpp = median_time(hmm.Evaluacion, s)
    tf_py  = median_time(nat.forward_log, s)
    tv_cpp = median_time(hmm.Reconocimiento, s)
    tv_py  = median_time(nat.viterbi, s)
    print(f"Secuencia con {n} letras: forward - libreria dinamica: {tf_cpp:.6f}s vs Py: {tf_py:.6f}s ({tf_py/tf_cpp:.2f}x), "
          f"viterbi -  libreria dinamica: {tv_cpp:.6f}s vs Py: {tv_py:.6f}s ({tv_py/tv_cpp:.2f}x)")


Secuencia con 1000 letras: forward - libreria dinamica: 0.000167s vs Py: 0.002180s (13.07x), viterbi -  libreria dinamica: 0.000083s vs Py: 0.001505s (18.11x)
Secuencia con 5000 letras: forward - libreria dinamica: 0.000828s vs Py: 0.010476s (12.66x), viterbi -  libreria dinamica: 0.000417s vs Py: 0.007363s (17.65x)
Secuencia con 10000 letras: forward - libreria dinamica: 0.001607s vs Py: 0.024499s (15.25x), viterbi -  libreria dinamica: 0.000848s vs Py: 0.014945s (17.63x)
Secuencia con 50000 letras: forward - libreria dinamica: 0.008195s vs Py: 0.149709s (18.27x), viterbi -  libreria dinamica: 0.004270s vs Py: 0.125975s (29.50x)


In [ ]:
import math

# Parámetros del modelo
PI = [0.5, 0.5]  # Inicio: H, L
A = [[0.5, 0.5],  # H -> H, L
     [0.4, 0.6]]  # L -> H, L
B = [[0.2, 0.3, 0.3, 0.2],  # H: A, C, G, T
     [0.3, 0.2, 0.2, 0.3]]  # L: A, C, G, T

IDX = {'A': 0, 'C': 1, 'G': 2, 'T': 3}

def manual_viterbi(seq):
    print("=== CÁLCULO MANUAL VITERBI ===")
    print(f"Secuencia: {seq}")
    print()

    n = len(seq)
    v = [[0.0, 0.0] for _ in range(n)]
    back = [[-1, -1] for _ in range(n)]

    # Inicialización
    print("Inicialización:")
    v[0][0] = math.log2(PI[0]) + math.log2(B[0][IDX[seq[0]]])
    v[0][1] = math.log2(PI[1]) + math.log2(B[1][IDX[seq[0]]])
    print(f"t=0, base={seq[0]} (índice {IDX[seq[0]]})")
    print(f"  H: log2(0.5) + log2({B[0][IDX[seq[0]]]}) = {math.log2(PI[0])} + {math.log2(B[0][IDX[seq[0]]])} = {v[0][0]}")
    print(f"  L: log2(0.5) + log2({B[1][IDX[seq[0]]]}) = {math.log2(PI[1])} + {math.log2(B[1][IDX[seq[0]]])} = {v[0][1]}")
    print()

    # Recursión
    for t in range(1, n):
        print(f"t={t}, base={seq[t]} (índice {IDX[seq[t]]})")

        # Estado H
        h_from_h = v[t-1][0] + math.log2(A[0][0])
        h_from_l = v[t-1][1] + math.log2(A[1][0])
        if h_from_h >= h_from_l:
            v[t][0] = h_from_h
            back[t][0] = 0
            print(f"  H: desde H ({h_from_h:.3f}) >= desde L ({h_from_l:.3f}) -> elegir H")
        else:
            v[t][0] = h_from_l
            back[t][0] = 1
            print(f"  H: desde H ({h_from_h:.3f}) < desde L ({h_from_l:.3f}) -> elegir L")

        v[t][0] += math.log2(B[0][IDX[seq[t]]])
        print(f"  H final: {v[t][0]:.3f}")

        # Estado L
        l_from_h = v[t-1][0] + math.log2(A[0][1])
        l_from_l = v[t-1][1] + math.log2(A[1][1])
        if l_from_h >= l_from_l:
            v[t][1] = l_from_h
            back[t][1] = 0
            print(f"  L: desde H ({l_from_h:.3f}) >= desde L ({l_from_l:.3f}) -> elegir H")
        else:
            v[t][1] = l_from_l
            back[t][1] = 1
            print(f"  L: desde H ({l_from_h:.3f}) < desde L ({l_from_l:.3f}) -> elegir L")

        v[t][1] += math.log2(B[1][IDX[seq[t]]])
        print(f"  L final: {v[t][1]:.3f}")
        print()

    # Backtracking
    last = 0 if v[n-1][0] >= v[n-1][1] else 1
    print(f"Estado final: {'H' if last == 0 else 'L'} (H: {v[n-1][0]:.3f}, L: {v[n-1][1]:.3f})")

    path = ['H'] * n
    for t in range(n-1, -1, -1):
        path[t] = 'H' if last == 0 else 'L'
        last = back[t][last]
        if t == 0:
            break

    resultado = ''.join(path)
    print(f"Camino más probable: {resultado}")
    print(f"Probabilidad final: {max(v[n-1]):.3f}")

    return resultado, max(v[n-1])

# Ejecutar para GGCACTGAA
resultado_manual, prob_manual = manual_viterbi("GGCACTGAA")


=== CÁLCULO MANUAL VITERBI ===
Secuencia: GGCACTGAA

Inicialización:
t=0, base=G (índice 2)
  H: log2(0.5) + log2(0.3) = -1.0 + -1.7369655941662063 = -2.7369655941662066
  L: log2(0.5) + log2(0.2) = -1.0 + -2.321928094887362 = -3.321928094887362

t=1, base=G (índice 2)
  H: desde H (-3.737) >= desde L (-4.644) -> elegir H
  H final: -5.474
  L: desde H (-3.737) >= desde L (-4.059) -> elegir H
  L final: -6.059

t=2, base=C (índice 1)
  H: desde H (-6.474) >= desde L (-7.381) -> elegir H
  H final: -8.211
  L: desde H (-6.474) >= desde L (-6.796) -> elegir H
  L final: -8.796

t=3, base=A (índice 0)
  H: desde H (-9.211) >= desde L (-10.118) -> elegir H
  H final: -11.533
  L: desde H (-9.211) >= desde L (-9.533) -> elegir H
  L final: -10.948

t=4, base=C (índice 1)
  H: desde H (-12.533) < desde L (-12.270) -> elegir L
  H final: -14.007
  L: desde H (-12.533) < desde L (-11.685) -> elegir L
  L final: -14.007

t=5, base=T (índice 3)
  H: desde H (-15.007) >= desde L (-15.329) -> eleg

In [ ]:
# Verificación: Forward vs Viterbi
import math

# Parámetros del modelo
PI = [0.5, 0.5]
A = [[0.5, 0.5], [0.4, 0.6]]
B = [[0.2, 0.3, 0.3, 0.2], [0.3, 0.2, 0.2, 0.3]]
IDX = {'A': 0, 'C': 1, 'G': 2, 'T': 3}

def forward_algorithm(seq):
    """Algoritmo Forward - probabilidad total de la secuencia"""
    if not seq:
        return float('-inf')

    n = len(seq)
    dp = [[0.0, 0.0] for _ in range(n)]

    def lse(a, b):
        if a == -math.inf:
            return b
        if b == -math.inf:
            return a
        m = max(a, b)
        return m + math.log2(math.exp2(a-m) + math.exp2(b-m))

    # Inicialización
    dp[0][0] = math.log2(PI[0]) + math.log2(B[0][IDX[seq[0]]])
    dp[0][1] = math.log2(PI[1]) + math.log2(B[1][IDX[seq[0]]])

    # Recursión
    for t in range(1, n):
        for s in (0, 1):
            fromH = dp[t-1][0] + math.log2(A[0][s])
            fromL = dp[t-1][1] + math.log2(A[1][s])
            dp[t][s] = lse(fromH, fromL) + math.log2(B[s][IDX[seq[t]]])

    return lse(dp[n-1][0], dp[n-1][1])

def viterbi_algorithm(seq):
    """Algoritmo Viterbi - camino más probable"""
    if not seq:
        return "", float('-inf')

    n = len(seq)
    v = [[0.0, 0.0] for _ in range(n)]
    back = [[-1, -1] for _ in range(n)]

    # Inicialización
    v[0][0] = math.log2(PI[0]) + math.log2(B[0][IDX[seq[0]]])
    v[0][1] = math.log2(PI[1]) + math.log2(B[1][IDX[seq[0]]])

    # Recursión
    for t in range(1, n):
        # Estado H
        h_from_h = v[t-1][0] + math.log2(A[0][0])
        h_from_l = v[t-1][1] + math.log2(A[1][0])
        if h_from_h >= h_from_l:
            v[t][0] = h_from_h
            back[t][0] = 0
        else:
            v[t][0] = h_from_l
            back[t][0] = 1
        v[t][0] += math.log2(B[0][IDX[seq[t]]])

        # Estado L
        l_from_h = v[t-1][0] + math.log2(A[0][1])
        l_from_l = v[t-1][1] + math.log2(A[1][1])
        if l_from_h >= l_from_l:
            v[t][1] = l_from_h
            back[t][1] = 0
        else:
            v[t][1] = l_from_l
            back[t][1] = 1
        v[t][1] += math.log2(B[1][IDX[seq[t]]])

    # Backtracking
    last = 0 if v[n-1][0] >= v[n-1][1] else 1
    path = ['H'] * n
    for t in range(n-1, -1, -1):
        path[t] = 'H' if last == 0 else 'L'
        last = back[t][last]
        if t == 0:
            break

    return ''.join(path), max(v[n-1])

# Probar ambos algoritmos
seq = "GGCACTGAA"
print("=== COMPARACIÓN: FORWARD vs VITERBI ===")
print(f"Secuencia: {seq}")
print()

# Forward (probabilidad total)
forward_prob = forward_algorithm(seq)
print(f"FORWARD (probabilidad total):")
print(f"  Log2 probabilidad: {forward_prob}")
print(f"  Probabilidad: {math.exp2(forward_prob)}")
print()

# Viterbi (camino más probable)
viterbi_path, viterbi_prob = viterbi_algorithm(seq)
print(f"VITERBI (camino más probable):")
print(f"  Camino: {viterbi_path}")
print(f"  Log2 probabilidad: {viterbi_prob}")
print(f"  Probabilidad: {math.exp2(viterbi_prob)}")
print()

print("EXPLICACIÓN:")
print("- Forward calcula la probabilidad TOTAL de la secuencia (suma de todos los caminos)")
print("- Viterbi calcula la probabilidad del camino MÁS PROBABLE")
print("- Forward siempre será mayor o igual que Viterbi")
print(f"- Diferencia: {forward_prob - viterbi_prob:.3f} en log2")


=== COMPARACIÓN: FORWARD vs VITERBI ===
Secuencia: GGCACTGAA

FORWARD (probabilidad total):
  Log2 probabilidad: -18.00898401038453
  Probabilidad: 3.7910160355199965e-06

VITERBI (camino más probable):
  Camino: HHHLLLLLL
  Log2 probabilidad: -24.487443319769202
  Probabilidad: 4.25152799999999e-08

EXPLICACIÓN:
- Forward calcula la probabilidad TOTAL de la secuencia (suma de todos los caminos)
- Viterbi calcula la probabilidad del camino MÁS PROBABLE
- Forward siempre será mayor o igual que Viterbi
- Diferencia: 6.478 en log2
